In [0]:
%sql
CREATE TABLE dim_entity (
    dim_key     BIGINT GENERATED ALWAYS AS IDENTITY,
    bk          STRING,
    attr1       STRING,
    attr2       STRING,
    valid_from  TIMESTAMP,
    valid_to    TIMESTAMP,
    is_current  BOOLEAN
);


In [0]:
MERGE INTO dim_entity AS t
USING src AS s
ON  t.bk = s.bk
AND t.is_current = true

WHEN MATCHED 
     AND (
          t.attr1 <> s.attr1
       OR t.attr2 <> s.attr2
       OR (t.attr1 IS NULL AND s.attr1 IS NOT NULL)
       OR (t.attr1 IS NOT NULL AND s.attr1 IS NULL)
       -- add comparisons for all tracked attributes
     )
THEN UPDATE SET
    t.valid_to   = s.etl_run_ts,   -- or s.row_last_updated_ts
    t.is_current = false

WHEN NOT MATCHED BY TARGET
THEN INSERT (bk, attr1, attr2, valid_from, valid_to, is_current)
VALUES (s.bk, s.attr1, s.attr2, s.etl_run_ts, TIMESTAMP '9999-12-31', true);


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit

dim_path = "/mnt/delta/dim_entity"

dim_delta = DeltaTable.forPath(spark, dim_path)

src_df = (
    src_raw_df
    # keep one row per bk (e.g., latest by timestamp)
)

cond = "t.bk = s.bk AND t.is_current = true"

change_expr = """
  t.attr1 <> s.attr1 OR t.attr2 <> s.attr2
  OR (t.attr1 IS NULL AND s.attr1 IS NOT NULL)
  OR (t.attr1 IS NOT NULL AND s.attr1 IS NULL)
"""

(dim_delta
 .alias("t")
 .merge(src_df.alias("s"), cond)
 .whenMatched(condition=change_expr)
 .update({
     "valid_to":   col("s.etl_run_ts"),
     "is_current": lit(False)
 })
 .whenNotMatched()
 .insert({
     "bk":         col("s.bk"),
     "attr1":      col("s.attr1"),
     "attr2":      col("s.attr2"),
     "valid_from": col("s.etl_run_ts"),
     "valid_to":   lit("9999-12-31"),
     "is_current": lit(True)
 })
 .execute()
)
